<a href="https://colab.research.google.com/github/mainulfikri08/Test-distillation/blob/main/Salinan_Distiller_Rembo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import tensorflow
from tensorflow.keras.layers import (
    LSTM,
    Dense,
    Flatten,
    Dropout,
    BatchNormalization,
    Bidirectional
    )

train = pd.read_csv('/content/drive/MyDrive/Dataset VAE/uncalibrated/train_final_u/train_4_u.csv')
test = pd.read_csv('/content/drive/MyDrive/Dataset VAE/uncalibrated/valid_final_u/valid_4_u.csv')


#del variabel yang tidak dipakai
del train['Timestamp']
del train['Cycle']
del train["Active Button"]

del test['Timestamp']
del test['Cycle']
del test["Active Button"]

print('Input train shape :', train.shape)
print('Input test shape :', test.shape)

Input train shape : (91000, 10)
Input test shape : (41600, 10)


In [ ]:
# Prepare the train and test dataset.

n_ts = 100 #bebas
n_outputs = 4
n_features = 10

x_train = (train.values[:,0:10])
xt = int(x_train.shape[0]/n_ts) #ubah ke 3D
print('x_train_samples =',xt)
x_train = x_train.reshape((xt, n_ts, 10))
x_train = x_train.astype('float32')
y_train = pd.read_csv('/content/drive/MyDrive/Dataset VAE/hasil_labeling_train_u.csv') #label train
y_train = y_train.astype('float32')

x_test = (test.values[:,0:10])
xs = int(x_test.shape[0]/n_ts)
print('x_test_samples =',xs)
x_test = x_test.reshape((xs,n_ts,10))
x_test = x_test.astype('float32')
y_test = pd.read_csv('/content/drive/MyDrive/Dataset VAE/hasil_labeling_valid_u.csv') #label test
y_test = y_test.astype('float32')

######################
y_train = y_train - 1
y_test = y_test - 1

#y_train = to_categorical(y_train)
#y_test = to_categorical(y_test)

print('Shape final')
x_train.shape,y_train.shape, x_test.shape, y_test.shape


x_train_samples = 910
x_test_samples = 416
Shape final


((910, 100, 10), (910, 1), (416, 100, 10), (416, 1))

In [ ]:
#Create the teacher
teacher = keras.Sequential(
    [

        keras.Input(shape=(n_ts,n_features)),
        layers.Conv1D(filters=64, kernel_size=3, activation='relu'),
        layers.Dropout(0.5),
        layers.MaxPooling1D(pool_size=2),
        layers.Flatten(),
        layers.Dense(100, activation='relu'),
        layers.Dense(n_outputs, activation='softmax'),
#        layers.LSTM(100, activation='tanh'),
#        layers.Dropout(0.5),
#        layers.Dense(100, activation='relu'),
#        layers.Dense(n_outputs, activation='softmax'),
    ],
    name="teacher",
)
teacher.summary()

Model: "teacher"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 98, 64)            1984      
                                                                 
 dropout (Dropout)           (None, 98, 64)            0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 49, 64)           0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 3136)              0         
                                                                 
 dense (Dense)               (None, 100)               313700    
                                                                 
 dense_1 (Dense)             (None, 4)                 404       
                                                           

In [ ]:
# Create the teacher
teacher = keras.Sequential(
    [
        keras.Input(shape=(n_ts,n_features)),
        layers.Bidirectional(LSTM(100, activation='tanh')),
        layers.Dropout(0.5),
        layers.Dense(100, activation='relu'),
        layers.Dense(n_outputs, activation='softmax'),
    ],
    name="teacher",
)
teacher.summary()

Model: "teacher"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 200)              88800     
 l)                                                              
                                                                 
 dropout_5 (Dropout)         (None, 200)               0         
                                                                 
 dense_10 (Dense)            (None, 100)               20100     
                                                                 
 dense_11 (Dense)            (None, 4)                 404       
                                                                 
Total params: 109,304
Trainable params: 109,304
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Create the student
student = keras.Sequential(
    [
        keras.Input(shape=(n_ts,n_features)),
        layers.LSTM(30, activation='tanh'),
        layers.Dropout(0.5),
        #layers.Dense(30, activation='relu'),
        layers.Dense(n_outputs, activation='softmax'),
    ],
    name="student",
)
student.summary()

Model: "student"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30)                4920      
                                                                 
 dropout_1 (Dropout)         (None, 30)                0         
                                                                 
 dense_2 (Dense)             (None, 4)                 124       
                                                                 
Total params: 5,044
Trainable params: 5,044
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Clone student for later comparison
student_scratch = keras.models.clone_model(student)

In [ ]:
# Train teacher as usual
teacher.compile(loss=tensorflow.keras.losses.sparse_categorical_crossentropy,
              optimizer=tensorflow.keras.optimizers.Adam(),
              metrics=[keras.metrics.SparseCategoricalAccuracy()])

# Train and evaluate teacher on data.
teacher.fit(x_train, y_train, epochs=300, batch_size=64)
score = teacher.evaluate(x_test, y_test, verbose=0)
print(f'Theacer loss: {score[0]}')
print(f'Teacher accuracy: {score[1]}')

Epoch 1/300
15/15 [==============================] - 8s 8ms/step - loss: 719.7944 - sparse_categorical_accuracy: 0.4747
Epoch 2/300
15/15 [==============================] - 0s 3ms/step - loss: 45.3666 - sparse_categorical_accuracy: 0.7615
Epoch 3/300
15/15 [==============================] - 0s 4ms/step - loss: 7.7461 - sparse_categorical_accuracy: 0.9242
Epoch 4/300
15/15 [==============================] - 0s 3ms/step - loss: 1.7108 - sparse_categorical_accuracy: 0.9912
Epoch 5/300
15/15 [==============================] - 0s 4ms/step - loss: 1.6520 - sparse_categorical_accuracy: 0.9890
Epoch 6/300
15/15 [==============================] - 0s 3ms/step - loss: 0.7881 - sparse_categorical_accuracy: 0.9923
Epoch 7/300
15/15 [==============================] - 0s 3ms/step - loss: 0.8393 - sparse_categorical_accuracy: 0.9934
Epoch 8/300
15/15 [==============================] - 0s 3ms/step - loss: 0.4909 - sparse_categorical_accuracy: 0.9934
Epoch 9/300
15/15 [==============================] - 

In [ ]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)

            # Compute scaled distillation loss from https://arxiv.org/abs/1503.02531
            # The magnitudes of the gradients produced by the soft targets scale
            # as 1/T^2, multiply them by T^2 when using both hard and soft targets.
            distillation_loss = (
                self.distillation_loss_fn(
                    tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                    tf.nn.softmax(student_predictions / self.temperature, axis=1),
                )
                * self.temperature**2
            )

            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results

In [ ]:
# Initialize and compile distiller
distiller = Distiller(student=student, teacher=teacher)
distiller.compile(
    optimizer=tensorflow.keras.optimizers.Adam(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=10,
)


# Distill teacher to student
distiller.fit(x_train, y_train, epochs=50, batch_size=64)

# Evaluate student on test dataset
score = distiller.evaluate(x_test, y_test, verbose=0)
print(f'Student loss: {score[1]}')
print(f'Student accuracy: {score[0]}')

Epoch 1/50


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


15/15 [==============================] - 2s 10ms/step - sparse_categorical_accuracy: 0.3242 - student_loss: 1.5422 - distillation_loss: 0.1039
Epoch 2/50
15/15 [==============================] - 0s 10ms/step - sparse_categorical_accuracy: 0.3538 - student_loss: 1.3686 - distillation_loss: 0.0963
Epoch 3/50
15/15 [==============================] - 0s 9ms/step - sparse_categorical_accuracy: 0.4154 - student_loss: 1.2590 - distillation_loss: 0.0884
Epoch 4/50
15/15 [==============================] - 0s 9ms/step - sparse_categorical_accuracy: 0.4659 - student_loss: 1.2012 - distillation_loss: 0.0842
Epoch 5/50
15/15 [==============================] - 0s 9ms/step - sparse_categorical_accuracy: 0.4593 - student_loss: 1.1859 - distillation_loss: 0.0839
Epoch 6/50
15/15 [==============================] - 0s 9ms/step - sparse_categorical_accuracy: 0.5231 - student_loss: 1.1126 - distillation_loss: 0.0783
Epoch 7/50
15/15 [==============================] - 0s 9ms/step - sparse_categorical_accura

/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


#Train student from scratch for comparison

In [ ]:
# Train student as doen usually
student_scratch.compile(loss=tensorflow.keras.losses.sparse_categorical_crossentropy,
              optimizer=tensorflow.keras.optimizers.Adam(),
              metrics=[keras.metrics.SparseCategoricalAccuracy()])


# Train and evaluate student trained from scratch.
student_scratch.fit(x_train, y_train, epochs=50, batch_size=64)
score = student_scratch.evaluate(x_test, y_test, verbose=0)
print(f'Base LSTM loss: {score[0]}')
print(f'Base LSTM accuracy: {score[1]}')

Epoch 1/50
15/15 [==============================] - 2s 9ms/step - loss: 1.5117 - sparse_categorical_accuracy: 0.3066
Epoch 2/50
15/15 [==============================] - 0s 9ms/step - loss: 1.4278 - sparse_categorical_accuracy: 0.3374
Epoch 3/50
15/15 [==============================] - 0s 8ms/step - loss: 1.3233 - sparse_categorical_accuracy: 0.3901
Epoch 4/50
15/15 [==============================] - 0s 8ms/step - loss: 1.3006 - sparse_categorical_accuracy: 0.3835
Epoch 5/50
15/15 [==============================] - 0s 8ms/step - loss: 1.2353 - sparse_categorical_accuracy: 0.4418
Epoch 6/50
15/15 [==============================] - 0s 8ms/step - loss: 1.1998 - sparse_categorical_accuracy: 0.4549
Epoch 7/50
15/15 [==============================] - 0s 7ms/step - loss: 1.1974 - sparse_categorical_accuracy: 0.4714
Epoch 8/50
15/15 [==============================] - 0s 8ms/step - loss: 1.1187 - sparse_categorical_accuracy: 0.5231
Epoch 9/50
15/15 [==============================] - 0s 7ms/step 